In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import ResNet50

# Load the CIFAR10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Convert the labels to categorical variables
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Normalize the input images
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255


In [2]:
# Create a CNN
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(x_train, y_train, epochs=20, batch_size=128, validation_data=(x_test, y_test))


Epoch 1/20
391/391 [==============================] - 15s 36ms/step - loss: 1.8600 - accuracy: 0.3044 - val_loss: 1.5013 - val_accuracy: 0.4671
Epoch 2/20
391/391 [==============================] - 13s 34ms/step - loss: 1.5196 - accuracy: 0.4475 - val_loss: 1.3634 - val_accuracy: 0.5047
Epoch 3/20
391/391 [==============================] - 14s 35ms/step - loss: 1.3904 - accuracy: 0.5016 - val_loss: 1.2347 - val_accuracy: 0.5623
Epoch 4/20
391/391 [==============================] - 17s 43ms/step - loss: 1.2954 - accuracy: 0.5401 - val_loss: 1.1631 - val_accuracy: 0.5865
Epoch 5/20
391/391 [==============================] - 14s 36ms/step - loss: 1.2320 - accuracy: 0.5647 - val_loss: 1.0934 - val_accuracy: 0.6098
Epoch 6/20
391/391 [==============================] - 15s 38ms/step - loss: 1.1615 - accuracy: 0.5900 - val_loss: 1.0642 - val_accuracy: 0.6235
Epoch 7/20
391/391 [==============================] - 15s 38ms/step - loss: 1.1128 - accuracy: 0.6106 - val_loss: 1.0233 - val_accuracy:

In [3]:
# Load the pre-trained ResNet50 model
resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Remove the last layer
resnet.layers.pop()

# Freeze the remaining layers
for layer in resnet.layers:
    layer.trainable = False

# Add additional layers
model_resnet = Sequential()
model_resnet.add(resnet)
model_resnet.add(Flatten())
model_resnet.add(Dense(64, activation='relu'))
model_resnet.add(Dropout(0.5))
model_resnet.add(Dense(10, activation='softmax'))

# Compile the model
model_resnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history_resnet = model_resnet.fit(x_train, y_train, epochs=20, batch_size=128, validation_data=(x_test, y_test))


Epoch 1/20
391/391 [==============================] - 80s 193ms/step - loss: 2.3184 - accuracy: 0.0991 - val_loss: 2.3026 - val_accuracy: 0.1000
Epoch 2/20
391/391 [==============================] - 96s 245ms/step - loss: 2.3027 - accuracy: 0.0975 - val_loss: 2.3026 - val_accuracy: 0.1000
Epoch 3/20
391/391 [==============================] - 116s 296ms/step - loss: 2.3027 - accuracy: 0.0988 - val_loss: 2.3026 - val_accuracy: 0.1000
Epoch 4/20
391/391 [==============================] - 130s 334ms/step - loss: 2.3027 - accuracy: 0.0960 - val_loss: 2.3026 - val_accuracy: 0.1000
Epoch 5/20
147/391 [==========>...................] - ETA: 1:09 - loss: 2.3027 - accuracy: 0.1004

KeyboardInterrupt: 

In [ ]:
# Evaluate the CNN model
_, acc_cnn = model.evaluate(x_test, y_test, verbose=0)
print(f"CNN Accuracy: {acc_cnn * 100:.2f}%")

# Evaluate the ResNet50 model
_, acc_resnet = model_resnet.evaluate(x_test, y_test, verbose=0)
print(f"ResNet50 Accuracy: {acc_resnet * 100:.2f}%")
